In [3]:
import os
import numpy as np
import scipy.io as sci_io
import matplotlib.pyplot as plt
import nitime.algorithms as tsa
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import warnings
import math
import neo as ne
import scipy.signal as signal
from pathlib import Path
warnings.filterwarnings('ignore')
import quantities as pq
from scipy.signal import hilbert, chirp, resample
from scipy.signal import butter, sosfilt, sosfreqz

In [18]:
DataDir = 'E:/Documents/ALESSANDRA/PROPAGATION/APP/Data/161218_APP_WT/161218_rec23_Propagation_125'
# DataDir = './frontalD300'

File1 = DataDir + '/Sig.mat'
mat_contents = sci_io.loadmat(File1)
Signal=mat_contents['S']
fs=4960
rs=np.array(Signal)

File2 = DataDir + '/Mua.mat'
mat_contents2 = sci_io.loadmat(File2)
Mua=mat_contents2['Mua']

fsMua=198.4127

MUA=np.zeros((32,np.shape(rs)[1]))
for i in range(32):
    MUA[i,:]=resample(Mua[i],np.shape(rs)[1])


In [19]:
analytic_signal=[]
amplitude_envelope=[]
instantaneous_phase=np.zeros((32,np.shape(rs)[1]))
iph=np.zeros((32,np.shape(rs)[1]))
instantaneous_frequency=np.zeros((32,np.shape(rs)[1]))

for ch in range(len(rs)):
    an_sig=hilbert(rs[ch])
#     analytic_signal=(an_sig)
#     amplitude_envelope=(np.abs(an_sig))
    instantaneous_phase[ch,:]=((np.angle(an_sig)))
    iph[ch,:]=np.unwrap(np.angle(an_sig))
#     instantaneous_frequency[ch,:] = (np.diff(iph) /(2.0*np.pi) * fs)


In [7]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [9]:
plt.figure(figsize=(15, 5))
ax=plt.subplot(211)
plt.plot(instantaneous_phase[0])
plt.plot(instantaneous_phase[6])
plt.plot(instantaneous_phase[14])
plt.plot(instantaneous_phase[31])
plt.subplot(212,sharex=ax)
plt.plot(MUA[0])
plt.plot(MUA[6])
plt.plot(MUA[14])
plt.plot(MUA[31])
plt.show()

In [10]:
B=np.arange(-math.pi,math.pi+0.15707963267948966,0.15707963267948966)

In [20]:
ph=np.empty((32))
H=np.zeros((np.shape(B)[0]-1,np.shape(rs)[1]))

for t in range(len(instantaneous_phase[0])):
    for ch in range(len(instantaneous_phase)):
#         print(ch,t)
        ph[ch]=instantaneous_phase[ch][t]
    hist, b = np.histogram(ph,bins=B)
    H[:,t]=hist
    ph=np.empty((32))
    

In [13]:
plt.figure(figsize=(15, 5))
ax1=plt.subplot(211)
ax=plt.imshow(H, aspect='auto')
ax.set_clim(0,15)
# plt.colorbar(ax)
plt.title('Phase Histogram')
plt.ylabel('phase bins')
plt.xlabel('samples')
plt.subplot(212,sharex=ax1)
plt.plot(MUA[0])
plt.plot(MUA[31])



In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(H.std(0))

In [ ]:
np.shape(instantaneous_phase)[1]

In [21]:
Dph=np.empty((496))
dH=np.zeros((np.shape(B)[0]-1,np.shape(rs)[1]))

n=0

for t in range(np.shape(instantaneous_phase)[1]):
    for i in range(0,32):
        for j in range(i+1,32):
            Dph[n]=(instantaneous_phase[i][t]-instantaneous_phase[j][t])
            n=n+1

    
    hist, b = np.histogram(Dph,bins=B)
    dH[:,t]=hist
    Dph=np.empty((496))
    n=0       
            

In [26]:
plt.figure(figsize=(15, 5))
ax=plt.subplot(311)
aa=plt.imshow(H, aspect='auto')
aa.set_clim(0,15)
# plt.colorbar(aa)
plt.title('Phase Histogram')
plt.ylabel('phase bins')
plt.xlabel('samples')
ax1=plt.subplot(312,sharex=ax)
ax1=plt.imshow(dH, aspect='auto')
ax1.set_clim(0,100)
# plt.colorbar(ax1)
plt.title('Phase difference Histogram')
plt.ylabel('phase bins')
plt.xlabel('samples')
ax2=plt.subplot(313,sharex=ax)
plt.plot(MUA[0])
plt.plot(MUA[31])
plt.xlim(0,496032)

(0, 496032)